🧵 1. Model dengan Functional API

In [16]:
from keras import layers, models, Input
from keras.datasets import fashion_mnist

# Load & preprocessing
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

# Build model
input_ = Input(shape=(28, 28))
x = layers.Flatten()(input_)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(64, activation="relu")(x)
output = layers.Dense(10, activation="softmax")(x)

model = models.Model(inputs=input_, outputs=output)

# Compile and train
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.7837 - loss: 0.6222 - val_accuracy: 0.8644 - val_loss: 0.3897
Epoch 2/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8611 - loss: 0.3856 - val_accuracy: 0.8722 - val_loss: 0.3627
Epoch 3/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8775 - loss: 0.3384 - val_accuracy: 0.8736 - val_loss: 0.3331
Epoch 4/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8859 - loss: 0.3075 - val_accuracy: 0.8782 - val_loss: 0.3423
Epoch 5/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8908 - loss: 0.2926 - val_accuracy: 0.8886 - val_loss: 0.3120


🏗️ 2. Model dengan Subclassing API

In [17]:
from keras import Model

class MyCustomModel(Model):
    def __init__(self):
        super().__init__()
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(128, activation="relu")
        self.dense2 = layers.Dense(64, activation="relu")
        self.output_layer = layers.Dense(10, activation="softmax")

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense1(x)
        x = self.dense2(x)
        return self.output_layer(x)

model = MyCustomModel()
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7717 - loss: 0.6440 - val_accuracy: 0.8524 - val_loss: 0.4156
Epoch 2/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8566 - loss: 0.3906 - val_accuracy: 0.8692 - val_loss: 0.3553
Epoch 3/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8745 - loss: 0.3443 - val_accuracy: 0.8846 - val_loss: 0.3284
Epoch 4/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8822 - loss: 0.3193 - val_accuracy: 0.8848 - val_loss: 0.3241
Epoch 5/5
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8889 - loss: 0.3023 - val_accuracy: 0.8884 - val_loss: 0.3130


🔁 3. Custom Training Loop dengan GradientTape

In [19]:
import tensorflow as tf

batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(batch_size)
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(batch_size)

model = MyCustomModel()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc = tf.keras.metrics.SparseCategoricalAccuracy()

for epoch in range(5):
    print(f"\nEpoch {epoch+1}")
    for X_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            loss = loss_fn(y_batch, y_pred)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc.update_state(y_batch, y_pred)
    print("Train Accuracy:", train_acc.result().numpy())
    train_acc.reset_state()


Epoch 1
Train Accuracy: 0.8149273

Epoch 2
Train Accuracy: 0.8641091

Epoch 3
Train Accuracy: 0.8766

Epoch 4
Train Accuracy: 0.8839818

Epoch 5
Train Accuracy: 0.89003634


## ✅ Kesimpulan Kode:

| Teknik          | Kegunaan Utama                                 |
| --------------- | ---------------------------------------------- |
| Functional API  | Model bercabang/kompleks tapi tetap high-level |
| Subclassing API | Model sangat fleksibel & OOP style             |
| GradientTape    | Kontrol penuh atas training loop               |
